# Libraries

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# $\mathcal{O}$ problem and $g^{-1}$ function

In [2]:
def Oracle(lambda_, lambda_prime, p, B):
    tol = 1e-8
    max_iter = 100

    def x_star(mu, p, lambda_, lambda_prime):
        # Formula aggiornata
        return np.maximum(0, (1 / lambda_) * np.log(p * lambda_prime / mu))

    def total_allocation(mu, p, lambda_, lambda_prime, B):
        return np.sum(x_star(mu, p, lambda_, lambda_prime)) - B

    def find_optimal_mu(p, lambda_, lambda_prime, B):
        mu_low = 1e-12
        mu_high = max(p * lambda_prime)

        for _ in range(max_iter):
            mu_mid = (mu_low + mu_high) / 2
            total = total_allocation(mu_mid, p, lambda_, lambda_prime, B)

            if abs(total) < tol:
                return mu_mid
            elif total > 0:
                mu_low = mu_mid
            else:
                mu_high = mu_mid
        return mu_mid

    mu_opt = find_optimal_mu(p, lambda_, lambda_prime, B)
    x_opt = x_star(mu_opt, p, lambda_, lambda_prime)
    return x_opt

def true_mu(x):
    return 1 / x - np.exp(-x) / (1 - np.exp(-x))

def g_inv(y, x0_guess=0.5):
    def invert_single_value(y0):
        solution = fsolve(lambda x: true_mu(x) - y0, x0_guess)[0]
        return solution if solution > 0 else np.nan

    if np.isscalar(y):
        return invert_single_value(y)
    else:
        y = np.asarray(y)
        return np.array([invert_single_value(yi) for yi in y])

# Parameters

In [3]:
K = 10
B = 40
D = 0.07
T = 10000

p_min, p_max = 0.01, 1.0
lambda_min, lambda_max = 1/B, 80/B

# Uniform Sampling
p = np.random.uniform(p_min, p_max, size=K)
lambda_ = np.random.uniform(lambda_min, lambda_max, size=K)

print(p)
print(lambda_)

# Optimal Allocation
x_star = Oracle(lambda_, lambda_, p, B)
print(x_star)


[0.71395425 0.35461783 0.70112486 0.7819909  0.79441712 0.01441034
 0.31644707 0.81046164 0.04610347 0.82216255]
[0.79499642 1.14444077 1.63997871 0.14415798 0.55303521 0.50213521
 1.1400584  1.39112281 1.57367185 1.94047303]
[ 4.61471437  2.91254651  2.66750379 14.23631063  6.17059306  0.
  2.82048328  3.13056142  1.02408908  2.42319786]


# Functions

In [4]:
def compute_confidence_bounds(t, i, p_hat, lambda_hat, n, x_est, B, D, K):
    p_bar = np.zeros(K)
    lambda_bar = np.zeros(K)
    lambda_bar_prime = np.zeros(K)
    for j in range(K):
        c_lambda = 1 / (B * D) * np.sqrt(3 * np.log(t + 1) / (2 * (n[t, j] + 1e-2)))
        c_p = 0.01*(1 + p_hat[t, j]) / ((1 - np.exp(-x_est[t, j] / B)) * D + 1e-2) * \
              np.sqrt(3 * np.log(t + 1) / (2 * (n[t, j] + 1e-2)))
        if j == i:
            lambda_bar[j] = max(1 / B, lambda_hat[t, j] - c_lambda)
            lambda_bar_prime[j] = max(1 / B, lambda_hat[t, j] + c_lambda)
            p_bar[j] = max(0.001, min(1, p_hat[t, j] + c_p))
        else:
            lambda_bar[j] = max(1 / B, lambda_hat[t, j] + c_lambda)
            lambda_bar_prime[j] = max(1 / B, lambda_hat[t, j] - c_lambda)
            p_bar[j] = max(0.001, min(1, p_hat[t, j] - c_p))
    return p_bar, lambda_bar , lambda_bar_prime

def select_allocation(lambda_bar, lambda_bar_prime, p_bar, B):
    return Oracle(lambda_bar, lambda_bar_prime, p, B)

def sample_outcomes(t1, lambda_, p, K, x_reward, X, Y, f, F, f_star, x_star, B):
    X[t1 + 1] = np.random.exponential(scale=1.0 / lambda_)
    Y[t1 + 1] = np.random.binomial(n=1, p=p)
    for k in range(K):
        if Y[t1 + 1, k] == 1 and x_reward[t1 + 1, k] >= X[t1 + 1, k]:
            f[t1 + 1, k] = 1
            F[t1 + 1, k] = X[t1 + 1, k]
        else:
            f[t1 + 1, k] = 0
            F[t1 + 1, k] = 10 * B
        if Y[t1 + 1, k] == 1 and x_star[k] >= X[t1 + 1, k]:
            f_star[t1 + 1, k] = 1
        else:
            f_star[t1 + 1, k] = 0

def update_estimates(t, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est):
    phi_val = phi[t + 1, i]
    psi_val = psi[t + 1, i]
    if phi_val == 1:
        n[t + 1, i] = n[t, i] + 1
        mu_hat[t + 1, i] = (mu_hat[t, i] * n[t, i] + psi_val) / n[t + 1, i]
    else:
        n[t + 1, i] = n[t, i]
        mu_hat[t + 1, i] = mu_hat[t, i]

    if mu_hat[t + 1, i] == 0:
        lambda_hat[t + 1, i] = 0
    else:
        lambda_hat[t + 1, i] = 1 / B * g_inv(min(0.5, mu_hat[t + 1, i] / B))

    denom = np.sum(1 - np.exp(-lambda_hat[t + 1, i] * x_est[:t + 2, i])) + 1e-6
    p_hat[t + 1, i] = np.sum(phi[:t + 2, i]) / denom

def compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star):
    reward[t1 + 1] = np.sum(f[t1 + 1])
    opt_reward[t1 + 1] = np.sum(f_star[t1 + 1])
    regret[t1 + 1] = opt_reward[t1 + 1] - reward[t1 + 1]
    cum_regret[t1 + 1] = np.sum(regret[:t1 + 2])

# $\texttt{RA-UCB}$ simulations

In [ ]:
num_experiments = 15
all_cum_regrets = []

for experiment in range(num_experiments):
    t1 = 0  #Reward time
    beta = 0.001
    init_steps_per_arm = int(beta * T / K)

    #Estimation Variables
    n = np.zeros((int(T/K), K))
    mu_hat = np.zeros((int(T/K), K))
    lambda_hat = np.zeros((int(T/K), K))
    p_hat = np.zeros((int(T/K), K))
    x_est = np.zeros((int(T/K), K))

    #Reward Variables
    x_reward = np.zeros((T, K))
    X = np.zeros((T, K))
    Y = np.zeros((T, K))
    F = np.zeros((T, K))
    f = np.zeros((T, K))
    f_star = np.zeros((T, K))
    reward = np.zeros(T)
    opt_reward = np.zeros(T)
    regret = np.zeros(T)
    cum_regret = np.zeros(T)

    phi = np.zeros((int(T/K), K))
    psi = np.zeros((int(T/K), K))

    # INITIALIZATION PHASE
    for i in range(K):
        for step in range(init_steps_per_arm):
            x_reward[t1 + 1] = np.zeros(K)
            x_reward[t1 + 1, i] = B

            X_sample = np.random.exponential(scale=1.0 / lambda_)
            Y_sample = np.random.binomial(n=1, p=p)
            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            phi[t1 // K + 1, i] = f[t1 + 1, i]
            psi[t1 // K + 1, i] = F[t1 + 1, i]
            update_estimates(t1 // K, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
            t1 += 1

    # MAIN PHASE
    for t in range(t1 // K, T // K - 1):  #Estimation time
        for i in range(K):
            X_sample = np.random.exponential(scale=1.0 / lambda_)
            Y_sample = np.random.binomial(n=1, p=p)

            p_bar, lambda_bar, lambda_bar_prime = compute_confidence_bounds(t, i, p_hat, lambda_hat, n, x_est, B, D, K)
            x_reward[t1 + 1] = select_allocation(lambda_bar, lambda_bar_prime, p_bar, B)
            x_est[t + 1, i] = x_reward[t1 + 1, i].copy()

            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            phi[t + 1, i] = f[t1 + 1, i]
            psi[t + 1, i] = F[t1 + 1, i]
            update_estimates(t, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)

            t1 += 1

    all_cum_regrets.append(cum_regret[:T])

all_cum_regrets = np.array(all_cum_regrets)


mean_regret = np.mean(all_cum_regrets, axis=0)   #Averaged cumulative regret values for different values of t
std_regret = np.std(all_cum_regrets, axis=0)     #Standard deviation of the cumulative regret

# Theoretical Upper Bound and Lower Bound

In [ ]:
q = np.zeros(K)
q = p*(1-np.exp(-lambda_*x_star))
G = np.sum(((1+p)*p)/(q**1.5))
V = np.sum(1/(lambda_*np.sqrt(q)))

#Upper Bound
def regret_ub(t):
  return 28/D * (G+np.sqrt(1/np.min(q))+1/(2*B)*V)*np.sqrt(K*t*np.log(t)) + np.sqrt(K*t) + 2*K**2*(4+np.log(t))

#Lower Bound
def regret_lb(t):
  return 1/48 * K**(2/3)*t**(1/3)

t_values = np.arange(2, T)
regret_ub_values = [regret_ub(t) for t in t_values]
regret_lb_values = [regret_lb(t) for t in t_values]

# Comparison between RA-UCB and RA-ETC

In [11]:
all_cum_regrets = []
all_cum_regrets_etc = []
num_experiments = 15

for experiment in range(num_experiments):

    t1 = 0
    t2 = 0

    # Generate the entire sequence of random samples
    X_stream = np.random.exponential(scale=1.0 / lambda_, size=(T, K))
    Y_stream = np.random.binomial(n=1, p=p, size=(T, K))

    # RA-UCB VARIABLES
    beta = 0.001
    init_steps_per_arm = int(beta * T / K)

    # Estimation (RA-UCB)
    n = np.zeros((int(T/K), K))
    mu_hat = np.zeros((int(T/K), K))
    lambda_hat = np.zeros((int(T/K), K))
    p_hat = np.zeros((int(T/K), K))
    x_est = np.zeros((int(T/K), K))
    phi = np.zeros((int(T/K), K))
    psi = np.zeros((int(T/K), K))

    # Reward (RA-UCB)
    x_reward = np.zeros((T, K))
    X = np.zeros((T, K))
    Y = np.zeros((T, K))
    F = np.zeros((T, K))
    f = np.zeros((T, K))
    f_star = np.zeros((T, K))
    reward = np.zeros(T)
    opt_reward = np.zeros(T)
    regret = np.zeros(T)
    cum_regret = np.zeros(T)

    # RA-ETC VARIABLES
    init_steps_per_arm_etc = int(T**(2/3))

    n3 = np.zeros((T, K))
    mu_hat3 = np.zeros((T, K))
    lambda_hat3 = np.zeros((T, K))
    lambda_hat3_def = np.zeros(K)
    p_hat3 = np.zeros((T, K))
    p_hat3_def = np.zeros(K)
    x_reward3 = np.zeros((T, K))
    X3 = np.zeros((T, K))
    Y3 = np.zeros((T, K))
    F3 = np.zeros((T, K))
    f3 = np.zeros((T, K))
    f_star3 = np.zeros((T, K))
    reward3 = np.zeros(T)
    opt_reward3 = np.zeros(T)
    regret3 = np.zeros(T)
    cum_regret3 = np.zeros(T)

    # INITIALIZATION PHASE RA-UCB
    for i in range(K):
        for step in range(init_steps_per_arm):
            x_reward[t1 + 1] = np.zeros(K)
            x_reward[t1 + 1, i] = B

            X_sample = X_stream[t1]
            Y_sample = Y_stream[t1]
            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B

                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            phi[t1 // K + 1, i] = f[t1 + 1, i]
            psi[t1 // K + 1, i] = F[t1 + 1, i]
            update_estimates(t1 // K, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
            t1 += 1

    # EXPLORATION PHASE RA-ETC
    for i in range(K):
        for step in range(init_steps_per_arm_etc):
            x_reward3[t2 + 1, i] = B
            X_sample = X_stream[t2]
            Y_sample = Y_stream[t2]
            X3[t2 + 1] = X_sample
            Y3[t2 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward3[t2 + 1, k] >= X_sample[k]:
                    f3[t2 + 1, k] = 1
                    n3[t2 + 1, k] = n3[t2, k] + 1
                    mu_hat3[t2 + 1, k] = (mu_hat3[t2, k] * n3[t2, k] + X_sample[k]) / n3[t2 + 1, k]
                else:
                    f3[t2 + 1, k] = 0
                    n3[t2 + 1, k] = n3[t2, k]
                    mu_hat3[t2 + 1, k] = mu_hat3[t2, k]

                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star3[t2 + 1, k] = 1
                else:
                    f_star3[t2 + 1, k] = 0

            for k in range(K):
                if mu_hat3[t2 + 1, k] == 0:
                    lambda_hat3[t2 + 1, k] = 0
                else:
                    lambda_hat3[t2 + 1, k] = 1 / B * g_inv(min(0.5, mu_hat3[t2 + 1, k] / B))
                denom = np.sum(1 - np.exp(-lambda_hat3[t2 + 1, k] * x_reward3[:t2 + 2, k])) + 1e-2
                p_hat3[t2 + 1, k] = np.sum(f3[:t2 + 2, k]) / denom

            lambda_hat3_def = lambda_hat3[t2 + 1].copy()
            p_hat3_def = p_hat3[t2 + 1].copy()

            reward3[t2 + 1] = np.sum(f3[t2 + 1])
            opt_reward3[t2 + 1] = np.sum(f_star3[t2 + 1])
            regret3[t2 + 1] = opt_reward3[t2 + 1] - reward3[t2 + 1]
            cum_regret3[t2 + 1] = np.sum(regret3[:t2 + 2])

            t2 += 1

    # MAIN PHASE RA-UCB
    for t in range(t1 // K, T // K - 1):
        for i in range(K):
            X_sample = X_stream[t1]
            Y_sample = Y_stream[t1]

            p_bar, lambda_bar, lambda_bar_prime = compute_confidence_bounds(t, i, p_hat, lambda_hat, n, x_est, B, D, K)
            x_reward[t1 + 1] = select_allocation(lambda_bar, lambda_bar_prime, p_bar, B)
            x_est[t + 1, i] = x_reward[t1 + 1, i].copy()

            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample
            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B

                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            phi[t + 1, i] = f[t1 + 1, i]
            psi[t + 1, i] = F[t1 + 1, i]
            update_estimates(t, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)

            t1 += 1

    # EXPLOITATION PHASE RA-ETC
    for t in range(t2 // K, T // K - 1):
        for i in range(K):
            X_sample = X_stream[t2]
            Y_sample = Y_stream[t2]

            x_reward3[t2 + 1] = Oracle(lambda_hat3_def, lambda_hat3_def, p_hat3_def, B)
            X3[t2 + 1] = X_sample
            Y3[t2 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward3[t2 + 1, k] >= X_sample[k]:
                    f3[t2 + 1, k] = 1
                else:
                    f3[t2 + 1, k] = 0

                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star3[t2 + 1, k] = 1
                else:
                    f_star3[t2 + 1, k] = 0

            reward3[t2 + 1] = np.sum(f3[t2 + 1])
            opt_reward3[t2 + 1] = np.sum(f_star3[t2 + 1])
            regret3[t2 + 1] = opt_reward3[t2 + 1] - reward3[t2 + 1]
            cum_regret3[t2 + 1] = np.sum(regret3[:t2 + 2])

            t2 += 1

    all_cum_regrets.append(cum_regret[:T])
    all_cum_regrets_etc.append(cum_regret3[:T])

all_cum_regrets = np.array(all_cum_regrets)
all_cum_regrets_etc = np.array(all_cum_regrets_etc)

# Average and Standard Deviation
mean_regret_ucb = np.mean(all_cum_regrets, axis=0)
std_regret_ucb = np.std(all_cum_regrets, axis=0)

mean_regret_etc = np.mean(all_cum_regrets_etc, axis=0)
std_regret_etc = np.std(all_cum_regrets_etc, axis=0)


# Comparison between $\texttt{RA-UCB}$ and $\texttt{NO UCB}$

In [ ]:
num_experiments = 15
all_cum_regrets = []
all_cum_regrets2 = []

for experiment in range(num_experiments):
  t1 = 0
  beta = 0.001
  init_steps_per_arm = int(beta * T / K)

  #Estimation Variables (RA-UCB)
  n = np.zeros((int(T/K), K))
  mu_hat = np.zeros((int(T/K), K))
  lambda_hat = np.zeros((int(T/K), K))
  p_hat = np.zeros((int(T/K), K))
  x_est = np.zeros((int(T/K), K))
  phi = np.zeros((int(T/K), K))
  psi = np.zeros((int(T/K), K))

  #Reward Variables (RA-UCB)
  x_reward = np.zeros((T, K))
  X = np.zeros((T, K))
  Y = np.zeros((T, K))
  F = np.zeros((T, K))
  f = np.zeros((T, K))
  f_star = np.zeros((T, K))
  reward = np.zeros(T)
  opt_reward = np.zeros(T)
  regret = np.zeros(T)
  cum_regret = np.zeros(T)

  # Variables (NO UCB)
  n2 = np.zeros((T, K))
  mu_hat2 = np.zeros((T, K))
  lambda_hat2 = np.zeros((T, K))
  p_hat2 = np.zeros((T, K))
  x_reward2 = np.zeros((T, K))
  X2 = np.zeros((T, K))
  Y2 = np.zeros((T, K))
  F2 = np.zeros((T, K))
  f2 = np.zeros((T, K))
  f_star2 = np.zeros((T, K))
  reward2 = np.zeros(T)
  opt_reward2 = np.zeros(T)
  regret2 = np.zeros(T)
  cum_regret2 = np.zeros(T)

  # INITIALIZATION PHASE
  for i in range(K):
      for step in range(init_steps_per_arm):
          x_reward[t1 + 1] = np.zeros(K)
          x_reward[t1 + 1, i] = B
          x_reward2[t1 + 1] = x_reward[t1 + 1].copy()

          X_sample = np.random.exponential(scale=1.0 / lambda_)
          Y_sample = np.random.binomial(n=1, p=p)

          X[t1 + 1] = X_sample
          Y[t1 + 1] = Y_sample
          X2[t1 + 1] = X_sample
          Y2[t1 + 1] = Y_sample

          for k in range(K):
              if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                  f[t1 + 1, k] = 1
                  F[t1 + 1, k] = X_sample[k]
              else:
                  f[t1 + 1, k] = 0
                  F[t1 + 1, k] = 10 * B

              if Y_sample[k] == 1 and x_reward2[t1 + 1, k] >= X_sample[k]:
                  f2[t1 + 1, k] = 1
                  n2[t1 + 1, k] = n2[t1, k] + 1
                  mu_hat2[t1 + 1, k] = (mu_hat2[t1, k] * n2[t1, k] + X_sample[k]) / n2[t1 + 1, k]
              else:
                  f2[t1 + 1, k] = 0
                  n2[t1 + 1, k] = n2[t1, k]
                  mu_hat2[t1 + 1, k] = mu_hat2[t1, k]

              if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                  f_star[t1 + 1, k] = 1
                  f_star2[t1 + 1, k] = 1
              else:
                  f_star[t1 + 1, k] = 0
                  f_star2[t1 + 1, k] = 0

          phi[t1 // K + 1, i] = f[t1 + 1, i]
          psi[t1 // K + 1, i] = F[t1 + 1, i]
          update_estimates(t1 // K, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)

          for k in range(K):
              if mu_hat2[t1 + 1, k] == 0:
                  lambda_hat2[t1 + 1, k] = 0
              else:
                  lambda_hat2[t1 + 1, k] = 1 / B * g_inv(min(0.5, mu_hat2[t1 + 1, k] / B))

              denom = np.sum(1 - np.exp(-lambda_hat2[t1 + 1, k] * x_reward2[:t1 + 2, k])) + 1e-2
              p_hat2[t1 + 1, k] = np.sum(f2[:t1 + 2, k]) / denom

          compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)

          reward2[t1 + 1] = np.sum(f2[t1 + 1])
          opt_reward2[t1 + 1] = np.sum(f_star2[t1 + 1])
          regret2[t1 + 1] = opt_reward2[t1 + 1] - reward2[t1 + 1]
          cum_regret2[t1 + 1] = np.sum(regret2[:t1 + 2])

          t1 += 1

  # MAIN PHASE
  for t in range(t1 // K, T // K - 1):
      for i in range(K):
          X_sample = np.random.exponential(scale=1.0 / lambda_)
          Y_sample = np.random.binomial(n=1, p=p)

          p_bar, lambda_bar, lambda_bar_prime = compute_confidence_bounds(t, i, p_hat, lambda_hat, n, x_est, B, D, K)
          x_reward[t1 + 1] = select_allocation(lambda_bar, lambda_bar_prime, p_bar, B)
          x_est[t + 1, i] = x_reward[t1 + 1, i].copy()

          X[t1 + 1] = X_sample
          Y[t1 + 1] = Y_sample
          for k in range(K):
              if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                  f[t1 + 1, k] = 1
                  F[t1 + 1, k] = X_sample[k]
              else:
                  f[t1 + 1, k] = 0
                  F[t1 + 1, k] = 10 * B

              if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                  f_star[t1 + 1, k] = 1
              else:
                  f_star[t1 + 1, k] = 0

          phi[t + 1, i] = f[t1 + 1, i]
          psi[t + 1, i] = F[t1 + 1, i]
          update_estimates(t, i, B, phi, psi, n, mu_hat, lambda_hat, p_hat, x_est)
          compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)

          lambda_bar2 = np.zeros(K)
          p_bar2 = np.zeros(K)

          for k in range(K):
              lambda_bar2[k] = max(1 / B, lambda_hat2[t1, k])
              p_bar2[k] = max(0.001, min(1, p_hat2[t1, k]))

          x_reward2[t1 + 1] = Oracle(lambda_bar2, lambda_bar2, p_bar2, B)
          X2[t1 + 1] = X_sample
          Y2[t1 + 1] = Y_sample

          for k in range(K):
              if Y_sample[k] == 1 and x_reward2[t1 + 1, k] >= X_sample[k]:
                  f2[t1 + 1, k] = 1
                  n2[t1 + 1, k] = n2[t1, k] + 1
                  mu_hat2[t1 + 1, k] = (mu_hat2[t1, k] * n2[t1, k] + X_sample[k]) / n2[t1 + 1, k]
              else:
                  f2[t1 + 1, k] = 0
                  n2[t1 + 1, k] = n2[t1, k]
                  mu_hat2[t1 + 1, k] = mu_hat2[t1, k]

              if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                  f_star2[t1 + 1, k] = 1
              else:
                  f_star2[t1 + 1, k] = 0

          for k in range(K):
              if mu_hat2[t1 + 1, k] == 0:
                  lambda_hat2[t1 + 1, k] = 0
              else:
                  lambda_hat2[t1 + 1, k] = 1 / B * g_inv(min(0.5, mu_hat2[t1 + 1, k] / B))

              denom = np.sum(1 - np.exp(-lambda_hat2[t1 + 1, k] * x_reward2[:t1 + 2, k])) + 1e-2
              p_hat2[t1 + 1, k] = np.sum(f2[:t1 + 2, k]) / denom

          reward2[t1 + 1] = np.sum(f2[t1 + 1])
          opt_reward2[t1 + 1] = np.sum(f_star2[t1 + 1])
          regret2[t1 + 1] = opt_reward2[t1 + 1] - reward2[t1 + 1]
          cum_regret2[t1 + 1] = np.sum(regret2[:t1 + 2])

          t1 += 1
  all_cum_regrets.append(cum_regret[:T])
  all_cum_regrets2.append(cum_regret2[:T])

all_cum_regrets = np.array(all_cum_regrets)
all_cum_regrets2 = np.array(all_cum_regrets2)

mean_regret = np.mean(all_cum_regrets, axis=0)
std_regret = np.std(all_cum_regrets, axis=0)

mean_regret2 = np.mean(all_cum_regrets2, axis=0)
std_regret2 = np.std(all_cum_regrets2, axis=0)